# Two-Way ANOVA with Dunnett and TukeyHSD Posthoc Comparisons 

Performs ANOVAs and Dunnett posthoc comparisons for ANOVAs using a Python wrapper for R. 
 

### User Input

In [4]:
f_in = "coupling_connected_nmda.csv"
exp_name = 'Demo'

### Import Statements for Python and Library loading for R

In [5]:

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import pandas as pd
import numpy as np
import os
from rpy2.robjects import r, pandas2ri
from IPython.display import display, HTML


base = importr('base')
utils = importr('utils')
desc_tools = importr('DescTools')
fsa = importr('FSA')
rcompanion = importr('rcompanion')
mcv = importr('multcompView')
lsmeans = importr('lsmeans')
stats = importr('stats')
ag = importr('agricolae')
dunnett = ro.r['DunnettTest']

### Read in the Data via Python

There are well-documented bugs in the transferring of a pandas dataframe to an R dataframe, so the offical way to ensure your analysis is formatted correctly, you have to parse out the columns into individual variables at the start.

In [19]:
# Read the clean data into a pandas dataframe
df_in = pd.read_csv(f_in)
df_r = ro.pandas2ri.py2ri (df_in)
print(type(df_r))

# Rearrange data for 2-way Anova (indexed)
df_control = df_in[['strain', "distance", "cc", "gj", "vm1", "rin1"]].copy()
df_control['treatment'] = 'control'
df_treated = df_in[['strain', "distance", "cc_nmda", "gj_nmda", "vm1_nmda", "rin1_nmda"]].copy()
df_treated.columns = ['strain', "distance", "cc", "gj", "vm1", "rin1"]
df_treated['treatment'] = 'nmda'

df = df_control.append(df_treated)
df['interaction'] = df.strain + " : " + df.treatment
display(HTML(df.to_html()))

# Parse the data in local R variables
# Pandas dataframes do not faithfully convert to R dataframes
cc = ro.vectors.FloatVector (df.cc)
gj = ro.vectors.FloatVector (df.gj)
vm = ro.vectors.FloatVector (df.vm1)
rin = ro.vectors.FloatVector (df.rin1)
dist = ro.vectors.FloatVector (df.distance)
strain = ro.vectors.FactorVector (df.strain)
treatment = ro.vectors.FactorVector (df.treatment)
interaction = ro.vectors.FactorVector (df.interaction)


# Convert local R variables to global R variables
# Does not work if you do it all in one step
ro.globalenv ['strain'] = strain
ro.globalenv ['cc'] = cc
ro.globalenv ['gj'] = gj
ro.globalenv ['vm'] = vm
ro.globalenv ['rin'] = rin
ro.globalenv ['dist'] = dist
ro.globalenv ['treatment'] = treatment
ro.globalenv ['interaction'] = interaction

# Make a list of measurements to be analyzed
analyze = [cc, gj, vm, rin]
labels = ["cc",  "gj", "vm", "rin"]

# Function for saving pandas dataframes to csv spreadsheets
def save_csv (df_out, file_out):
    df_out.to_csv(file_out, index = False)

<class 'rpy2.robjects.vectors.DataFrame'>


,strain,distance,cc,gj,vm1,rin1,treatment,interaction
0,group1,30,0.182209,17.390415,-41.7200,127.383696,control,group1 : control
1,group4,20,0.102526,3.696705,-32.8793,371.505858,control,group4 : control
2,group2,0,0.326218,25.075822,-47.0485,286.589037,control,group2 : control
3,group4,5,0.146041,9.138495,-31.8213,256.237951,control,group4 : control
4,group1,5,0.108131,5.456852,-38.2370,136.785980,control,group1 : control
5,group3,15,0.155002,17.590365,-46.8462,87.931161,control,group3 : control
6,group2,30,0.092935,4.037475,-60.4135,31.015254,control,group2 : control
7,group4,10,0.146109,5.559933,-35.9055,228.972417,control,group4 : control
8,group2,5,0.117745,11.052959,-49.1488,117.128402,control,group2 : control
9,group4,20,0.357832,24.687704,-42.2800,84.119238,control,group4 : control


### Two-Way ANOVA with Mouse Strain and Treatment as Factors

In [7]:
def get_anova (dep_var, label, df_anovas):
    """User passes the the dependent variable, the group label, and output dataframe (pandas).
        The function calculates the linear model, summarizes the data, then performs an 
        anova."""
    df_anova = pd.DataFrame()
    display(HTML('<h4> Two-Way ANOVA Results: ' + label))
    ro.globalenv['dep_var'] = dep_var
    lm_x = stats.lm("dep_var ~ strain + treatment +  strain:treatment")
    lm_summary = base.summary(lm_x)
    anova_results = stats.anova(lm_x)
    anova_summary = base.summary(anova_results)
    
    # Parse the chaotic output of lm() and anova() into a pandas dataframe
    df_strain = int(anova_results[0][0])
    df_treatment =int(anova_results[0][1])
    df_interaction = int(anova_results[0][2])
    df_res = int(anova_results[0][3])
    ss_strain = format(round(anova_results[1][0],3), 'f')
    ss_treatment = format(round(anova_results[1][1],3), 'f')
    ss_interaction = format(round(anova_results[1][2],3), 'f')
    ss_res = format(round(anova_results[1][3],3), 'f')
    ms_strain = anova_results[2][0]
    ms_treatment = anova_results[2][1]
    ms_interaction = anova_results[2][2]
    ms_res = anova_results[2][3]
    f_strain = anova_results[3][0]
    f_treatment = anova_results[3][1]
    f_interaction = anova_results[3][2]
    f_res = anova_results[3][3]
    p_strain = anova_results[4][0]
    p_treatment = anova_results[4][1]
    p_interaction = anova_results[4][2]
    p_res = anova_results[4][3]
    
    # Assemble the rows for the dataframe
    row1 = {'measure': label, 'comparison': 'strain', 'df': df_strain, 'ss': ss_strain,
           'ms': ms_strain, 'f_stat': f_strain, 'p_val': p_strain}
    row2 = {'measure': label, 'comparison': 'treatment', 'df': df_treatment, 'ss': ss_treatment,
           'ms': ms_treatment, 'f_stat': f_treatment, 'p_val': p_treatment}
    row3 = {'measure': label, 'comparison': 'interaction', 'df': df_interaction, 'ss': ss_interaction,
           'ms': ms_interaction, 'f_stat': f_interaction, 'p_val': p_interaction}
    row4 = {'measure': label, 'comparison': 'residual', 'df': df_res, 'ss': ss_res,
           'ms': ms_res, 'f_stat': f_res, 'p_val': p_res}
    
    # Append the rows to the dataframe
    df_anova = df_anova.append([row1,row2,row3,row4], ignore_index = True, sort = False) 
    df_anovas = df_anovas.append(df_anova)
    display(HTML(df_anova.to_html()))

    return df_anovas

# Create and empty pandas dataframe to catch the output of the get_anova1() function


# Cycle through the measurements and perform one way anova on each with strain as the ind. var

df_anovas = pd.DataFrame()
i = 0
while i < len(analyze):
    df_anovas = get_anova(analyze[i], labels[i], df_anovas)
    i = i + 1   
    
save_csv (df_anovas, exp_name + '_two_way_anova.csv' ) # Save the dataframe to a csv spreadsheet



,measure,comparison,df,ss,ms,f_stat,p_val
0,cc,strain,3,8.178000,2.726006,5.699429,0.001076
1,cc,treatment,1,0.010000,0.009609,0.020090,0.887510
2,cc,interaction,3,0.440000,0.146592,0.306489,0.820661
3,cc,residual,128,61.222000,0.478295,NaN,NaN


,measure,comparison,df,ss,ms,f_stat,p_val
0,gj,strain,3,84565.907000,28188.635677,7.781918,0.000082
1,gj,treatment,1,159.408000,159.408216,0.044007,0.834174
2,gj,interaction,3,3881.099000,1293.699559,0.357146,0.784047
3,gj,residual,128,463657.620000,3622.325159,NaN,NaN


,measure,comparison,df,ss,ms,f_stat,p_val
0,vm,strain,3,753.876000,251.292128,2.316299,0.078794
1,vm,treatment,1,417.229000,417.229019,3.845831,0.052041
2,vm,interaction,3,208.737000,69.579096,0.641349,0.589778
3,vm,residual,128,13886.547000,108.488646,NaN,NaN


,measure,comparison,df,ss,ms,f_stat,p_val
0,rin,strain,3,128220.429000,42740.143067,2.640003,0.052298
1,rin,treatment,1,14137.086000,14137.085935,0.873229,0.351822
2,rin,interaction,3,13179.933000,4393.311127,0.271369,0.845951
3,rin,residual,128,2072247.181000,16189.431098,NaN,NaN


### Dunnett Pairwise Comparisons with Strain as Factor

In [8]:
 def get_dunnett(dep_var, label, df_dun):
        df_dun = pd.DataFrame()
        display(HTML('<h4> Dunnett Planned Comparisons: ' + label))
        dunnett_results = dunnett(x = dep_var, g = interaction, control = 'group1 : control')
        pairs = base.labels(dunnett_results[0])[0]
        calculations = base.labels(dunnett_results[0])[1]
        col_length = len(np.unique(strain)) + len(calculations) - 1
        i = 0
        for pair in pairs:
            groups = pair.split('-')
            row = {'measurement': label,
                   'group1': groups[0],
                   'group2': groups[1],
                   'diff': dunnett_results[0][i],
                   'lwr_ci': dunnett_results[0][i+col_length],
                   'upr_ci': dunnett_results[0][i+col_length*2],
                   'p_val': dunnett_results[0][i+col_length*3]}
            df_dun = df_dun.append(row, ignore_index = True, sort = False)
            df_dun = df_dun[['measurement','group1','group2', 'diff', 'lwr_ci', 'upr_ci', 'p_val' ]]
            i = i + 1
        display(HTML(df_dun.to_html()))
        return df_dun 

# Create an empty pandas dataframe to catch the data from the get_dunnett1 function

df_duns = pd.DataFrame()
i = 0
while i < len(analyze)-1:
    df_duns = get_dunnett(analyze[i], labels[i], df_duns)
    i = i + 1   




,measurement,group1,group2,diff,lwr_ci,upr_ci,p_val
0,cc,group1 : nmda,group1 : control,-0.099877,-0.657459,0.457706,0.997739
1,cc,group2 : control,group1 : control,-0.496233,-1.093409,0.100942,0.149650
2,cc,group2 : nmda,group1 : control,-0.433227,-1.030402,0.163949,0.266590
3,cc,group3 : control,group1 : control,-0.797724,-1.561225,-0.034222,0.036323
4,cc,group3 : nmda,group1 : control,-0.521212,-1.284713,0.242289,0.328724
5,cc,group4 : control,group1 : control,-0.594873,-1.159054,-0.030692,0.033993
6,cc,group4 : nmda,group1 : control,-0.592148,-1.156329,-0.027967,0.034976


,measurement,group1,group2,diff,lwr_ci,upr_ci,p_val
0,gj,group1 : nmda,group1 : control,2.428757,-46.095068,50.952583,1.000000
1,gj,group2 : control,group1 : control,-48.860113,-100.829534,3.109308,0.075452
2,gj,group2 : nmda,group1 : control,-47.110315,-99.079736,4.859106,0.094059
3,gj,group3 : control,group1 : control,-67.021467,-133.465451,-0.577483,0.047171
4,gj,group3 : nmda,group1 : control,-38.313358,-104.757343,28.130626,0.515214
5,gj,group4 : control,group1 : control,-50.665678,-99.763770,-1.567586,0.040063
6,gj,group4 : nmda,group1 : control,-58.551604,-107.649695,-9.453512,0.011361


,measurement,group1,group2,diff,lwr_ci,upr_ci,p_val
0,vm,group1 : nmda,group1 : control,4.992982,-3.404585,13.390548,0.480961
1,vm,group2 : control,group1 : control,-1.678176,-10.672040,7.315688,0.997091
2,vm,group2 : nmda,group1 : control,-1.780964,-10.774828,7.212899,0.995823
3,vm,group3 : control,group1 : control,4.039972,-7.458870,15.538813,0.910309
4,vm,group3 : nmda,group1 : control,5.236934,-6.261908,16.735776,0.751443
5,vm,group4 : control,group1 : control,1.091369,-7.405581,9.588318,0.999731
6,vm,group4 : nmda,group1 : control,6.831097,-1.665852,15.328047,0.174977


### Tukey HSD Posthoc Test

In [9]:
def get_tukey(dep_var, label, df_tukey):
    lm_x = stats.lm("dep_var ~ strain + treatment +  strain:treatment")
    tukey_aov = stats.aov(lm_x)
    tukey_results = stats.TukeyHSD(tukey_aov)
    calculations = base.labels(tukey_results[0])[1]
    col_length = int(len(tukey_results[2])/len(calculations))
    comparisons = base.labels(tukey_results[2])[0]

    i = 0
    while i < col_length:
        comparison = comparisons[i].split('-')
        row = {'group1': comparison[0],
               'group2': comparison[1],
                'diff': tukey_results[2][i],
                'lwr' : tukey_results[2][i + col_length],
                'upr' : tukey_results[2][i + col_length*2],
                'p_adj' : tukey_results[2][i + col_length*3]}

        df_tukey = df_tukey.append(row, ignore_index = True)
        df_tukey = df_tukey[['group1', 'group2', 'diff', 'upr', 'p_adj']]
        i = i + 1
    return df_tukey
    


i = 0
while i < len(analyze)-1:
    df_tukey = pd.DataFrame()
    display(HTML('<h4> Tukey HSD: ' + labels[i]))
    df_tukey = get_tukey (analyze[i], labels[i], df_tukey)
    display(HTML(df_tukey.to_html()))
    i = i + 1   
    


,group1,group2,diff,upr,p_adj
0,group2:control,group1:control,-11.617049,115.002990,0.999993
1,group3:control,group1:control,4.857697,166.744048,1.000000
2,group4:control,group1:control,46.783943,166.408180,0.929056
3,group1:nmda,group1:control,-46.023070,72.202005,0.930688
4,group2:nmda,group1:control,-33.009163,93.610876,0.992668
5,group3:nmda,group1:control,9.274428,171.160780,1.000000
6,group4:nmda,group1:control,44.605150,164.229387,0.944370
7,group3:control,group2:control,16.474746,184.589756,0.999988
8,group4:control,group2:control,58.400991,186.328408,0.852608
9,group1:nmda,group2:control,-34.406021,92.214018,0.990589


,group1,group2,diff,upr,p_adj
0,group2:control,group1:control,-11.617049,115.002990,0.999993
1,group3:control,group1:control,4.857697,166.744048,1.000000
2,group4:control,group1:control,46.783943,166.408180,0.929056
3,group1:nmda,group1:control,-46.023070,72.202005,0.930688
4,group2:nmda,group1:control,-33.009163,93.610876,0.992668
5,group3:nmda,group1:control,9.274428,171.160780,1.000000
6,group4:nmda,group1:control,44.605150,164.229387,0.944370
7,group3:control,group2:control,16.474746,184.589756,0.999988
8,group4:control,group2:control,58.400991,186.328408,0.852608
9,group1:nmda,group2:control,-34.406021,92.214018,0.990589


,group1,group2,diff,upr,p_adj
0,group2:control,group1:control,-11.617049,115.002990,0.999993
1,group3:control,group1:control,4.857697,166.744048,1.000000
2,group4:control,group1:control,46.783943,166.408180,0.929056
3,group1:nmda,group1:control,-46.023070,72.202005,0.930688
4,group2:nmda,group1:control,-33.009163,93.610876,0.992668
5,group3:nmda,group1:control,9.274428,171.160780,1.000000
6,group4:nmda,group1:control,44.605150,164.229387,0.944370
7,group3:control,group2:control,16.474746,184.589756,0.999988
8,group4:control,group2:control,58.400991,186.328408,0.852608
9,group1:nmda,group2:control,-34.406021,92.214018,0.990589


### Reference code for remotely installing R packages through python

In [11]:
#utils.chooseCRANmirror(ind=1) 
#utils.install_packages ("DescTools")
#utils.install_packages("rcompanion")
#utils.install_packages("lsmeans")
#utils.install_packages("multcompView")
#utils.install_packages("FSA")
#utils.install_packages("agricolae")